<a href="https://colab.research.google.com/github/puerchik1/ipynd/blob/main/Itogovoe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PySpark**


In [36]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, to_date, sum

spark = SparkSession.builder\
.appName('itogovoe')\
.config("spark.master","local[*]")\
.getOrCreate()

df = spark.read.csv('/content/web_server_logs.csv', header=True,inferSchema=True)
# df.printSchema()
# df.show(10)

# Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP
top = df.groupBy('ip')\
.agg(count('ip').alias('request_count'))\
.orderBy('request_count', ascending=False)\
.limit(10)
print("Top 10 active IP addresses:")
top.show()

# Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода
http_count = df.groupBy('method')\
.agg(count('ip').alias('method_count'))
print("Reques count by HTTP method:")
http_count.show()

# Профильтруйте и посчитайте количество запросов с кодом ответа 404
sum_code = df.filter(col('response_code') == 404).count()
print(f'Number of 404 response codes: {sum_code}')

# Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате
date_df = df.withColumn('date', to_date(col('timestamp')))
res = date_df.groupBy('date')\
.agg(sum('response_size').alias('total_response_size'))\
.orderBy('date', ascending=True)
print('Total response size by day:')
res.show()

spark.stop()

Top 10 active IP addresses:
+--------------+-------------+
|            ip|request_count|
+--------------+-------------+
|84.209.132.105|            2|
| 154.199.29.41|            1|
| 162.174.8.116|            1|
|24.165.103.123|            1|
|40.172.168.206|            1|
|194.98.173.250|            1|
|  29.211.75.38|            1|
|  5.150.42.167|            1|
|  14.16.71.110|            1|
|  5.26.199.137|            1|
+--------------+-------------+

Reques count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       24992|
|DELETE|       25063|
|   PUT|       24901|
|   GET|       25044|
+------+------------+

Number of 404 response codes: 24881
Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2025-01-01|            8445824|
|2025-01-02|            8374681|
|2025-01-03|            8548525|
|2025-01-04|            9039563|
|2025-01-05|            9037872|
|20